### 3-1

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# --- 중요 수정 사항 1: OpenAI 임베딩 및 LLM 임포트 경로 변경 ---
# 기존: from langchain.embeddings import OpenAIEmbeddings
# 기존: from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
# -------------------------------------------------------------

from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

# -----------------------------------------------------------------------------
# 0. 환경 설정 (Environment Setup)
#    - .env 파일에서 API 키 로드
# -----------------------------------------------------------------------------
print("0. 환경 설정 시작...")
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다. .env 파일을 확인하거나, API 키를 직접 코드에 설정하세요.")

print("환경 설정 완료.")

# -----------------------------------------------------------------------------
# 1. 문서 로딩 (Document Loading)
#    - '콘텐츠분쟁해결_사례.pdf' 파일 로드
# -----------------------------------------------------------------------------
print("\n1. 문서 로딩 시작...")

# --- 중요 수정 사항 2: PDF 파일 경로 수정 ---
# PDF 파일이 'C:\mylangchain\langchain_basic\data' 폴더에 있다고 가정
pdf_filepath = r'C:\mylangchain\langchain_basic\data\콘텐츠분쟁해결_사례.pdf'
# -------------------------------------------------------------

if not os.path.exists(pdf_filepath):
    print(f"오류: '{pdf_filepath}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해주세요.")
    exit()

try:
    # --- 중요 수정 사항 3: PyPDFLoader에 file_path 인자 전달 ---
    loader = PyPDFLoader(pdf_filepath)
    # -------------------------------------------------------------
    documents = loader.load()
    print(f"'{pdf_filepath}'에서 {len(documents)} 페이지 문서를 성공적으로 로드했습니다.")
except Exception as e:
    print(f"PDF 로딩 중 오류 발생: {e}")
    exit()

# -----------------------------------------------------------------------------
# 2. 텍스트 분할 (Text Splitting)
#    - 로드된 문서를 작은 청크로 분할
# -----------------------------------------------------------------------------
print("\n2. 텍스트 분할 시작...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)
texts = text_splitter.split_documents(documents)
print(f"문서를 {len(texts)}개의 청크로 분할했습니다.")

# -----------------------------------------------------------------------------
# 3. 임베딩 생성 (Generating Embeddings)
#    - 분할된 텍스트 청크를 벡터로 변환
# -----------------------------------------------------------------------------
print("\n3. 임베딩 생성 시작...")
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")
print("OpenAI 임베딩 모델 로드 완료.")

# -----------------------------------------------------------------------------
# 4. 벡터 스토어 구축 (Building Vector Store - FAISS)
#    - 임베딩된 청크를 FAISS에 저장하여 검색 가능하게 함
# -----------------------------------------------------------------------------
print("\n4. 벡터 스토어 구축 시작 (FAISS)...")
vector_store = FAISS.from_documents(texts, embeddings_model)
print("FAISS 벡터 스토어 구축 완료.")

# -----------------------------------------------------------------------------
# 5. 검색기 설정 (Setting up Retriever)
#    - 벡터 스토어를 기반으로 한 검색기 생성
# -----------------------------------------------------------------------------
print("\n5. 검색기 설정 시작...")
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
print("검색기 설정 완료 (상위 3개 문서 검색).")

# -----------------------------------------------------------------------------
# 6. 언어 모델 (LLM) 설정 (Setting up LLM)
#    - 질문 답변에 사용할 LLM 모델 설정
# -----------------------------------------------------------------------------
print("\n6. 언어 모델 (LLM) 설정 시작...")
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0.1)
print("LLM (gpt-3.5-turbo-0125) 설정 완료.")

# -----------------------------------------------------------------------------
# 7. RAG 체인 구축 및 실행 (Building and Running RAG Chain)
#    - 검색된 문서와 질문을 LLM에 전달하여 답변 생성
# -----------------------------------------------------------------------------
print("\n7. RAG 체인 구축 및 실행 시작...")

prompt_template = """다음 맥락을 사용하여 질문에 답변하세요.
만약 맥락에서 답변을 찾을 수 없다면, 모른다고 말하고 질문을 다시 작성하도록 요청하세요.
불필요한 정보를 추가하지 마세요. 질문과 맥락을 기반으로 답변을 생성하세요.

맥락:
{context}

질문:
{question}

답변:"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
print("RAG 체인 구축 완료.")

print("\n-------------------------------------------------------------")
print("RAG 시스템이 준비되었습니다. 질문을 입력해주세요. (종료: 'exit')")
print("-------------------------------------------------------------")

while True:
    query = input("\n질문: ")
    if query.lower() == 'exit':
        print("RAG 시스템을 종료합니다.")
        break

    print("답변 생성 중...")
    try:
        result = qa_chain.invoke({"query": query})
        answer = result["result"]
        source_docs = result["source_documents"]

        print(f"\n답변:\n{'-'*50}\n{answer}\n")

        print("참조 문서:")
        for i, doc in enumerate(source_docs[:3]):
            page_num = doc.metadata.get('page', 'N/A')
            source_file = doc.metadata.get('source', 'N/A')
            # 페이지 내용의 일부만 보여줌
            preview = doc.page_content.replace('\n', ' ')[:100] + "..."
            print(f"  {i+1}. 페이지 {page_num} (파일: {os.path.basename(source_file)}): {preview}")

    except Exception as e:
        print(f"RAG 체인 실행 중 오류 발생: {e}")
        print("API 키 또는 네트워크 연결을 확인해주세요.")